In [ ]:
from flask import Flask, render_template, redirect, url_for, request, flash, session
import pandas as pd
from flask_sqlalchemy import SQLAlchemy
from flask_login import UserMixin, login_user, LoginManager, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
from flask_bcrypt import check_password_hash
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, BooleanField, TextAreaField
from wtforms.validators import InputRequired, Length, ValidationError
from wtforms.fields.html5 import DateField
from datetime import datetime

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///user.db'
app.config['SECRET_KEY'] = "thisisasecretkey"
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

class User(db.Model, UserMixin):
    __tablename__ = 'user'
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    username = db.Column(db.String(20), unique=True, nullable = False)
    password = db.Column(db.String(80), nullable = False)
    email = db.Column(db.String(50), nullable = False)

class LoginForm(FlaskForm):
    username = StringField(label=u'用戶名稱', validators=[InputRequired(), Length(max=20)], render_kw={"placeholder":"用戶名稱"})
    password = PasswordField(label=u'密碼', validators=[InputRequired(), Length(max=20)], render_kw={"placeholder": "密碼"})
    submit = SubmitField("登入")
    
class Product(db.Model):
    __tablename__ = 'product'
    shop = db.Column(db.String(25))
    name = db.Column(db.Text, primary_key=True)
    oriprice = db.Column(db.Text)
    price = db.Column(db.Text)
    offer = db.Column(db.Text)
    shop_url = db.Column(db.Text)
    img_url = db.Column(db.Text)
    desc = db.Column(db.Text)

class Subemail(db.Model):
    __tablename__ = 'subemail'
    subemail = db.Column(db.String(50), primary_key=True, nullable = False)
    
@app.route("/", methods=["GET", "POST"])
def index():
    status = "home"
    if request.method == "POST":
        subemail = request.form['subemail']
        print(subemail)
        sub_email = Subemail(subemail=subemail)     
        try:
            db.session.add(sub_email)
            db.session.commit()
            print("Subscribed!")
            return redirect(url_for('index'))
        except Exception as e:
            print("Exception occurred.",e)
            return 'Fail to subscribed.'
    else:
        return render_template('index.html',status=status)

@app.route("/member", methods=["GET", "POST"])
def member():
    status = "memberhome"
    if request.method == "POST":
        subemail = request.form['subemail']
        print(subemail)
        sub_email = Subemail(subemail=subemail)     
        try:
            db.session.add(sub_email)
            db.session.commit()
            print("Subscribed!")
            return redirect(url_for('member'))
        except Exception as e:
            print("Exception occurred.",e)
            return 'Fail to subscribed.'
    else:
        return render_template('member.html',status=status)

@app.route("/register", methods=["GET", "POST"])
def register():
    status = "signup"
    if request.method == "POST":
        username = request.form['username']
        email = request.form['email']
        password = request.form['password']
        print(username,email,password)
        password = bcrypt.generate_password_hash(password)
        print(username,email,password)
        new_user = User(username=username,email=email,password=password)     
        try:
            db.session.add(new_user)
            db.session.commit()
            print("Registered!")
            return redirect(url_for('login'))
        except Exception as e:
            print("Exception occurred.",e)
            return 'Fail to add user.'
    else:
        return render_template('register.html',status=status)

@app.route("/login", methods=["GET", "POST"])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user:
            if bcrypt.check_password_hash(user.password, form.password.data):
                login_user(user)
                return redirect(url_for('member'))
        flash("User does not exist, or invalid username or password.")
    return render_template('login.html', title="Login", form=form)

@app.route("/logout", methods=["GET", "POST"])
@login_required
def logout():
    session.clear()
    logout_user()
    return redirect(url_for('index'))

@app.route("/product")
@login_required
def product():
    sqlalchemyObj = db.engine.execute('select * from product ORDER BY Name')
    product = []
    for i in sqlalchemyObj:
        product.append(i)
    dataset = []
    dict={}
    for i in product:
        dict['Shop'] = i[0]
        dict['Name'] = i[1]
        dict['Oriprice'] = i[2]
        dict['Price'] = i[3]
        dict['Shop_url'] = i[4]
        dict['Img_url'] = i[5]
        dict['Desc'] = i[6]
        # print(i)
        # print(dict)
        dataset.append(dict.copy()) 
    print(dataset)

    return render_template('product.html', title='Product', dataset=dataset)

if __name__ == '__main__':
    app.run('localhost', 9004)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


C:\Users\User\anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:9004/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Nov/2021 21:11:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2021 21:11:14] "GET /static/img/hero-bg.jpg HTTP/1.1" 404 -
127.0.0.1 - - [20/Nov/2021 21:11:15] "GET /static/img/footer-bg.jpg HTTP/1.1" 404 -
